In [ ]:
import sys # required for relative imports in jupyter lab
sys.path.insert(0, '../') 

from cosmosis.model import tv_model
from cosmosis.learning import Learn, Selector
from cosmosis.dataset import LoadImage, AsTensor, Transpose

from dataset import Ranzcr
from learning import RSelector

from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms

from IPython.display import display


In [ ]:
#TODO turn targets into ints also with none category
#re-balanced datasets
#create models for each target_type
#accuracy metric AUC for each category averaged see kaggle
#dataset normalization

ds_params={'train_params': {'embed': [],
                            'image_dir': './data/ranzcr/train',
                            'target_csv': './data/ranzcr/train.csv',
                            'target_type': 'ETT',
                            'transform': transforms.Compose([LoadImage(),
                                                             transforms.RandomRotation(10),
                                                             transforms.Resize(1500),
                                                             transforms.CenterCrop(1200),
                                                             transforms.Resize(300),
                                                             transforms.RandomCrop(244),
                                                             transforms.ToTensor(),
                                                             ]),
                            'target_transform': transforms.Compose([AsTensor(),
                                                                   ])},
           'val_params': {'embed': [],
                          'image_dir': './data/ranzcr/train',
                          'target_csv': './data/ranzcr/train.csv',
                          'target_type': 'ETT',
                          'transform': transforms.Compose([LoadImage(),
                                                           transforms.Resize(1500),
                                                           transforms.CenterCrop(1200),
                                                           transforms.Resize(300),
                                                           transforms.CenterCrop(244),
                                                           transforms.ToTensor(),
                                                           ]),
                          'target_transform': transforms.Compose([AsTensor(),
                                                                  ])},
           'test_params': {'embed': [],
                           'image_dir': './data/ranzcr/train',
                           'target_csv': './data/ranzcr/train.csv',
                           'target_type': 'ETT',
                           'transform': transforms.Compose([LoadImage(),
                                                            transforms.Resize(1500),
                                                            transforms.CenterCrop(1200),
                                                            transforms.Resize(300),
                                                            transforms.CenterCrop(244),
                                                            transforms.ToTensor(),
                                                            ]),
                          'target_transform': transforms.Compose([AsTensor(),
                                                                  ])}}

model_params = {'model_name': 'resnet18',
                'in_channels': 1,
                'tv_params': {'num_classes': 9}} 
    
opt_params = {'lr': 0.01}
crit_params = {'reduction': 'sum'}
sample_params = {'set_seed': False,
                 'splits': (.7,.15)}
sched_params = {'factor': .5,
                'patience': 2,
                'cooldown': 2}

l = Learn([Ranzcr,Ranzcr,Ranzcr], tv_model, RSelector, 
          Optimizer=Adam, Scheduler=ReduceLROnPlateau, Criterion=CrossEntropyLoss, 
          model_params=model_params, ds_params=ds_params, sample_params=sample_params,
          opt_params=opt_params, sched_params=sched_params, crit_params=crit_params,
          adapt=False, load_model=False, load_embed=False, save_model=True,
          batch_size=32, epochs=10)

In [ ]:
ds_params={'train_params': {'embed': [],
                            'image_dir': './data/ranzcr/train',
                            'target_csv': './data/ranzcr/train.csv',
                            'target_type': None,
                            'transform': transforms.Compose([LoadImage(),
                                                             #transforms.RandomRotation(10),
                                                             #transforms.Resize((1500)),
                                                             #transforms.CenterCrop(1200),
                                                             #transforms.Resize((300)),
                                                             #transforms.CenterCrop(244),
                                                             #transforms.ToTensor(),
                                                             #transforms.Normalize(0.3,0.3),
                                                             #transforms.ToPILImage(),
                                                             ]),
                            'target_transform': AsTensor()}}

ranzcr = Ranzcr(**ds_params['train_params'])

In [ ]:
ranzcr.ds_idx[0]

In [ ]:
ranzcr['12976413966762576562651637188467814800']

In [ ]:
display(ranzcr['12976413966762576562651637188467814800'][0])

In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv('./data/ranzcr/train.csv', header=0, index_col=0)

In [11]:
df.describe()

,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
count,30083.000000,30083.000000,30083.000000,30083.000000,30083.000000,30083.000000,30083.000000,30083.000000,30083.000000,30083.000000,30083.000000
mean,0.002626,0.037829,0.240667,0.009274,0.017585,0.091347,0.159459,0.106206,0.281222,0.708839,0.027590
std,0.051179,0.190785,0.427496,0.095857,0.131438,0.288107,0.366110,0.308106,0.449603,0.454305,0.163799
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df.loc['1.2.826.0.1.3680043.8.498.12976413966762576562651637188467814800']

In [ ]:
print(df.shape)
df.iloc[0]

In [ ]:
pos = df.loc[(df.iloc[:,3] == 1) & (df.iloc[:,4] == 0) & (df.iloc[:,5] == 0) & (df.iloc[:,6] == 1)]
pos
#neg = df.loc[df.iloc[:,:3] == 0].sample(pos.shape[0])
#neg.shape


In [ ]:
pos

In [25]:
import numpy as np

In [26]:
df = pd.read_csv('./data/ranzcr/train.csv', header=0, index_col=0)

In [44]:

s0 = list(df.loc[(df.iloc[:,0] == 1)].index.to_numpy())
cut1 = int(len(s0)*.7)
cut2 = int(len(s0)*.15)
s0train = s0[:cut1]
s0train = np.random.choice(s0train, 128, replace=True)
s0val = s0[cut1:cut1+cut2]
s0test = s0[cut1+cut2:]

s1 = list(df.loc[(df.iloc[:,1] == 1)].index.to_numpy())
cut1 = int(len(s1)*.7)
cut2 = int(len(s1)*.15)
s1train = s1[:cut1]
s1train = np.random.choice(s1train, 256, replace=False)
s1val = s1[cut1:cut1+cut2]
s1test = s1[cut1+cut2:]

s2 = list(df.loc[(df.iloc[:,2] == 1)].index.to_numpy())
cut1 = int(len(s2)*.7)
cut2 = int(len(s2)*.15)
s2train = s2[:cut1]
s2train = np.random.choice(s2train, 512, replace=False)
s2val = s2[cut1:cut1+cut2]
s2test = s2[cut1+cut2:]

s3 = list(df.loc[(df.iloc[:,0] == 0) & (df.iloc[:,1] == 0) & (df.iloc[:,2] == 0)].index.to_numpy())
cut1 = int(len(s3)*.7)
cut2 = int(len(s3)*.15)
s3train = s3[:cut1]
s3train = np.random.choice(s3train, 1024, replace=False)
s3val = s3[cut1:cut1+cut2]
s3test = s3[cut1+cut2:]

train_idx = np.concatenate([s0train,s1train,s2train,s3train])
val_idx = np.concatenate([s0val,s1val,s2val,s3val])
test_idx = np.concatenate([s0test,s1test,s2test,s3test])


1920 4510 4516
79 1138 7240 21626


In [45]:
train_idx[:20]

array(['1.2.826.0.1.3680043.8.498.13141026863513235286827713303306203465',
       '1.2.826.0.1.3680043.8.498.10336813034626604482298505167326531069',
       '1.2.826.0.1.3680043.8.498.11515952990369223019049010515619916078',
       '1.2.826.0.1.3680043.8.498.13355346266941406291822060610556042880',
       '1.2.826.0.1.3680043.8.498.41044962160142145688786829579416924470',
       '1.2.826.0.1.3680043.8.498.11333894316143324337529593966002928477',
       '1.2.826.0.1.3680043.8.498.77971501424098028244346066190633475857',
       '1.2.826.0.1.3680043.8.498.12546492988826740238933256204661756580',
       '1.2.826.0.1.3680043.8.498.62439834649842886640660835690358991386',
       '1.2.826.0.1.3680043.8.498.90015812224915102704830196980092330554',
       '1.2.826.0.1.3680043.8.498.10933179563635489243925857859639203436',
       '1.2.826.0.1.3680043.8.498.13141026863513235286827713303306203465',
       '1.2.826.0.1.3680043.8.498.12546492988826740238933256204661756580',
       '1.2.826.0.1.36800

In [33]:
l = list(ds.index.to_numpy())

In [34]:
l[0]

'1.2.826.0.1.3680043.8.498.64512214019606325590696659869142698791'

In [10]:
print(ett_0.shape, ett_1.shape, ett_2.shape, ett_3.shape)

(79, 12) (1138, 12) (7240, 12) (21626, 12)


In [ ]:
self.train_idx = self.dataset_idx[:cut1]
self.val_idx = self.dataset_idx[cut1:cut1+cut2]
self.test_idx = self.dataset_idx[cut1+cut2:]